In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv3D, MaxPool3D, Conv2D, MaxPool2D
from tensorflow.keras.layers import Flatten

In [5]:
from tensorflow.keras.backend import manual_variable_initialization 
manual_variable_initialization(True)

In [4]:
input = np.zeros([110000,32,3])
for rec in range(3):
    print(rec)
    ifile = pd.read_csv('raw data/raw data/range_' + str(10) + '_'+ str(rec)+ '.csv',header=None)
    for i in range(ifile.shape[0]):
        for j in range(ifile.shape[1]):
            c = complex(ifile.iloc[i,j])
            input[i,j,rec] = np.abs(c)
            
input1 = np.zeros([100000,32,3])
cnt = 0
x_range = [12,13,14,15,16,19,20,21,22,23]
for x in x_range:
    for rec in range(3):
        print(rec)
        ifile = pd.read_csv('raw data/raw data/range_' + str(x) + '_'+ str(rec)+ '.csv',header=None)
        for i in range(ifile.shape[0]):
            for j in range(ifile.shape[1]):
                c = complex(ifile.iloc[i,j])
                input1[cnt*10000+i,j,rec] = np.abs(c)
    cnt +=1
    
input2 = np.zeros([100000,32,3])
cnt = 0
x_range = [24,25,26]
for x in x_range:
    for rec in range(3):
        print(rec)
        ifile = pd.read_csv('raw data/raw data/range_' + str(x) + '_'+ str(rec)+ '.csv',header=None)
        for i in range(ifile.shape[0]):
            for j in range(ifile.shape[1]):
                c = complex(ifile.iloc[i,j])
                input2[cnt*10000+i,j,rec] = np.abs(c)
    cnt +=1

0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2


In [6]:
#Three set of experimental runs
#total of 23 experiments
data = np.zeros([10,10000,32,3])
for i in range(1,11):
    data[i-1,:,:,:] = input[i * 10000:(i+1)*10000,:,:]
    
data21 = np.zeros([10,10000,32,3])
for i in range(0,10):
    data21[i,:,:,:] = input1[i * 10000:(i+1)*10000,:,:]
    
data31 = np.zeros([3,10000,32,3])
for i in range(0,3):
    data31[i,:,:,:] = input2[i * 10000:(i+1)*10000,:,:]
    
data_full = np.concatenate([data,data21,data31],axis=0)
#The data is downsampled to have 1000 frames
data22 = data_full[:,0:-1:10,:,:]
label = np.array([1,0,0,1,1,0,1,1,0,1, 0,0,0,1,1,0,0,0,1,1,0,1,1])
# Scaling the input data to -1 to 1
data22 = data22*20

#The data from each reciever is set as different datapoint (Data augmentation)
data33 = np.zeros([69,1000,32,1])
for i in range(3):
    data33[i*23:(i+1)*23,:,:,:] =data22[:,:,:,i:i+1] 
    
label1 = np.concatenate([label,label,label],axis=0)

train, test, train_label,test_label = train_test_split(data33,label1, test_size=0.2, random_state=0, shuffle =True)

# Model building

In [2]:

def model_builder(n_filter,n_hidden,neuron,act,kernel):
    model = Sequential()

    n_conv = 2
    for j in range(n_conv):
        model.add(Conv2D(filters=n_filter,kernel_size=(32,kernel),activation='relu',padding='valid'))
        model.add(Conv2D(filters=n_filter,kernel_size=(32,kernel),activation='relu',padding='valid'))
        #model.add(MaxPool2D((8,2)))
    model.add(Flatten())
    for i in range(n_hidden):
        model.add(Dense(neuron,activation=act))

    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(optimizer=tf.keras.optimizers.Adam(0.0001),loss='binary_crossentropy', metrics= ['accuracy'])    
    return model

In [19]:
call = [tf.keras.callbacks.EarlyStopping(mode='min',monitor='val_loss',patience=10),
           tf.keras.callbacks.ModelCheckpoint('./tmp',monitor='val_loss',save_best_only=True)]

In [105]:
with tf.device('CPU:0'):
    model1 = model_builder(32,4,400,'relu',3)
    model1.fit(train,train_label, validation_data = (test,test_label), epochs=20, batch_size= 8, callbacks = call)

Epoch 1/10


TypeError: 'NoneType' object is not callable

In [22]:
model1.save_weights('2302/')

In [6]:
model1 = model_builder(32,4,400,'relu',3)
model1.load_weights('2302/')